In [1]:
import numpy as np
import matchzoo as mz

/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=True)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8038.16it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4314.91it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 2102259.30it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8377.25it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4329.35it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 565303.32it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 7538.04it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 4229.93it/s]
Processing length_right with len: 100%|██████████| 1115/1115 [00:00<00:00, 471626.56it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 8518.65it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 4507.65it/s]
Processing length_right with len: 100%|██████████| 2300/2300 [00:00<00:00, 564245.14it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
model = mz.models.MatchPyramid()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['embedding_input_dim'] = preprocessor.context['vocab_size'] + 1
model.params['embedding_output_dim'] = 300
model.params['embedding_trainable'] = True
model.params['num_blocks'] = 2
model.params['kernel_count'] = [16, 32]
model.params['kernel_size'] = [[3, 3], [3, 3]]
model.params['dpool_size'] = [3, 10]
model.params['optimizer'] = 'adam'
model.params['dropout_rate'] = 0.1
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to MatchPyramid.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 40)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             5002200     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
matching_layer_1 (MatchingLayer (None, 10, 40, 1)    0           embedd

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [8]:
model.load_embedding_matrix(embedding_matrix)

In [9]:
train_generator = mz.DPoolPairDataGenerator(train_pack_processed,
                                            fixed_length_left=10,
                                            fixed_length_right=40,
                                            num_dup=2,
                                            num_neg=1,
                                            batch_size=20)
len(train_generator)

102

In [10]:
predict_generator = mz.DPoolDataGenerator(predict_pack_processed,
                                          fixed_length_left=10,
                                          fixed_length_right=40,
                                          batch_size=20)
len(predict_generator)

118

In [11]:
pred_x, pred_y = predict_generator[:]
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_y))

In [12]:
history = model.fit_generator(train_generator, epochs=20, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/20
102/102 [==============================] - 23s 230ms/step - loss: 0.7675
Validation: normalized_discounted_cumulative_gain@3(0): 0.5666880709477722 - normalized_discounted_cumulative_gain@5(0): 0.6347123122214711 - mean_average_precision(0): 0.5849244632869276
Epoch 2/20
102/102 [==============================] - 22s 220ms/step - loss: 0.3962
Validation: normalized_discounted_cumulative_gain@3(0): 0.5740812009556767 - normalized_discounted_cumulative_gain@5(0): 0.643243583254445 - mean_average_precision(0): 0.5979056923205268
Epoch 3/20
102/102 [==============================] - 22s 217ms/step - loss: 0.2405
Validation: normalized_discounted_cumulative_gain@3(0): 0.5906235981391689 - normalized_discounted_cumulative_gain@5(0): 0.6488355555551298 - mean_average_precision(0): 0.5919047037059867
Epoch 4/20
102/102 [==============================] - 21s 206ms/step - loss: 0.1129
Validation: normalized_discounted_cumulative_gain@3(0): 0.5770363843119413 - normalized_discounted_cu